In [22]:
from PIL import Image, ImageDraw, ImageFont

def getNodePosition(i, nodes, deforms):
    """Получение новой позиции узла с учетом перемещений."""
    x, y = nodes[i]
    return (
        x + deforms[2 * i] * 50.0,  # перемещение по оси X
        y + deforms[2 * i + 1] * 50.0  # перемещение по оси Y
    )

def draw_legend(draw, image, min_value, max_value, label="σ_xx (MPa)"):
    """Рисует легенду для цветовой карты."""
    legend_x = image.size[0] - 150
    legend_y = 100
    legend_height = 800
    steps = 50

    for i in range(steps):
        v = i / steps
        colorf = ((v - 0.5) * 4.0, 2.0 - abs(v - 0.5) * 4.0, 2.0 - v * 4.0)
        colori = (int(colorf[0] * 255), int(colorf[1] * 255), int(colorf[2] * 255))
        y_pos = legend_y + (1 - v) * legend_height
        draw.rectangle([(legend_x, y_pos), (legend_x + 30, y_pos + legend_height / steps)], fill=colori)

    font = ImageFont.truetype("arial.ttf", 20)  
    draw.text((legend_x, legend_y - 20), f"{max_value:.2f} MPa", fill=(255, 255, 255), font=font)
    draw.text((legend_x, legend_y + legend_height + 5), f"{min_value:.2f} MPa", fill=(255, 255, 255), font=font)
    draw.text((legend_x - 40, legend_y + legend_height + 25), label, fill=(255, 255, 255), font=font)

def PostProcessSigmaXX(inputfile, sigmaXXfile, deformations_file, output_dir):
    """Визуализация компоненты σ_xx с учетом перемещений узлов."""
    with open(inputfile) as f:
        content = f.readlines()

    line = 1
    nodesCount = int(content[line])
    line += 1

    nodes = [(float(content[i].split()[0]), float(content[i].split()[1])) for i in range(line, line + nodesCount)]
    line += nodesCount

    min_x, min_y = max_x, max_y = nodes[0]
    for x, y in nodes:
        min_x, max_x = min(min_x, x), max(max_x, x)
        min_y, max_y = min(min_y, y), max(max_y, y)

    elementsCount = int(content[line])
    line += 1
    elements = [tuple(map(int, content[i].split())) for i in range(line, line + elementsCount)]

    center = [(max_x + min_x) / 2.0, (max_y + min_y) / 2.0]
    scale = max(max_x - min_x, max_y - min_y)

    # Чтение значений σ_xx (напряжения) из файла
    with open(sigmaXXfile) as f:
        sigmaXX_values = list(map(float, f.readlines()))

    # Чтение перемещений узлов из файла
    with open(deformations_file) as f:
        deformations = list(map(float, f.readlines()))

    min_sigmaXX, max_sigmaXX = min(sigmaXX_values), max(sigmaXX_values)
    print(f"Max σ_xx: {max_sigmaXX:.2f} MPa")
    print(f"Min σ_xx: {min_sigmaXX:.2f} MPa")

    def Transform(point):
        """Преобразование координат в масштабированные пиксели для изображения."""
        x = (point[0] - center[0]) / scale / 1.5 * image.size[0] + image.size[0] / 2.0
        y = (point[1] - center[1]) / scale / 1.5 * image.size[1] + image.size[1] / 2.0
        return (x, image.size[1] - y)

    image = Image.new("RGB", (1024, 1024))
    draw = ImageDraw.Draw(image)
    draw.rectangle([(0, 0), image.size], fill=0x777777)

    if max_sigmaXX == min_sigmaXX:
        max_sigmaXX += 1.0
        min_sigmaXX -= 1.0

    for idx, element in enumerate(elements):
        # Отображение напряжений с учетом перемещений узлов
        v = (sigmaXX_values[idx] - min_sigmaXX) / (max_sigmaXX - min_sigmaXX)
        colorf = ((v - 0.5) * 4.0, 2.0 - abs(v - 0.5) * 4.0, 2.0 - v * 4.0)
        colori = (int(colorf[0] * 255), int(colorf[1] * 255), int(colorf[2] * 255))
        
        # Используем новые позиции узлов с учетом перемещений
        deformed_nodes = [getNodePosition(element[i], nodes, deformations) for i in range(3)]
        draw.polygon([Transform(deformed_nodes[i]) for i in range(3)], fill=colori)

    draw_legend(draw, image, min_sigmaXX, max_sigmaXX, label="σ_xx (MPa)")
    image.save(output_dir + "/simgaXX.png", "PNG")

# Пример использования:
# PostProcessSigmaXX("mesh_data.txt", "sigmaXX_results.txt", "deformations.txt", "sigmaXX_map_with_deformations.png")


In [25]:
directory =  "C:/fydesis_cmake/result/"

inputfile = directory + "input_check.txt"  # Путь к файлу с узлами и элементами
sigmaXXfile = directory + "sigmaXX.txt"  # Путь к файлу с деформациями и напряжениями
outputfile = directory + "output.txt"  # Путь к файлу с деформациями и напряжениями
output_folder = directory  # Папка для сохранения изображений
PostProcessSigmaXX(inputfile, sigmaXXfile, outputfile, output_folder)


Max σ_xx: 311.13 MPa
Min σ_xx: 0.98 MPa
